In [17]:
import numpy as np
import pandas as pd
import mne
import csv, json
import datetime

In [9]:
sample_folder = './samples/'
file_name = 'Veh70_02202015'

In [6]:
def edf_to_pandas(edf_path, scores_path):
    edf = mne.io.read_raw_edf(edf_path)
    sampling_rate = int(edf.info['sfreq'])
    name = datetime.datetime.utcfromtimestamp(edf.info['meas_date'][0]).strftime('%Y-%m-%d %H:%M:%S')
    
    header = (','.join(edf.ch_names)).split(',')
    df = edf.get_data().T
    df = pd.DataFrame(data=df, columns=header)
    
    scores = pd.read_excel(scores_path, keep_default_na=False)
    repeat_score = int(df.shape[0]/scores.shape[0])
    scores = np.repeat(scores.values, repeat_score)
    
    scored_rows = min(df.shape[0], scores.shape[0])
    df = df[:scored_rows]
    
    df['Score'] = scores
    df.index.name = name
    
    return df

In [18]:
df = edf_to_pandas(sample_folder+file_name+'.edf', sample_folder+file_name+'.xls')

Extracting EDF parameters from C:\Users\Usuario\AutoScorer\model\samples\Veh70_02202015.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [19]:
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
X = df[["EEG","EMG"]]
X = scale(X) 
Y=df["Score"]
X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, train_size=0.75, test_size=0.25, random_state=0)

In [20]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
print('Accuracy of GNB classifier on training set: {:.2f}'
     .format(gnb.score(X_train, Y_train)))
print('Accuracy of GNB classifier on test set: {:.2f}'
     .format(gnb.score(X_test, Y_test)))

Accuracy of GNB classifier on training set: 0.45
Accuracy of GNB classifier on test set: 0.45


In [25]:
YPred = gnb.predict(X)
Predicted_Data=X
Predicted_Data["Predicted_Score"]=YPred

In [46]:
df.to_csv("./naive_model_output/Predicted_Data.csv")